In [5]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch

In [78]:
new_tokens = ['<detect>',*[f'<loc{v:04d}>' for v in range(1024)]]
len(new_tokens), new_tokens[:5]

(1025, ['<detect>', '<loc0000>', '<loc0001>', '<loc0002>', '<loc0003>'])

In [77]:
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-256M-Instruct", additional_special_tokens=new_tokens)

In [6]:
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
)

config.json:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [10]:
im = torch.rand(3,512,512)

In [40]:
a=processor(images=[im],do_rescale=False,do_normalize=False,return_tensors='pt',max_image_size={'longest_edge':512})

In [37]:
import matplotlib.pyplot as plt

In [46]:
model.resize_token_embeddings(len(processor.tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50305, 576, padding_idx=2)

In [69]:
messages1 = [{'role': 'user', 'content': [{'type': 'image'}, {'type': 'text', 'text': '<|detect|> person; paddle'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|loc0082|><|loc0137|><|loc1023|><|loc0857|>; person\n<|loc0537|><|loc0212|><|loc1000|><|loc0440|>; person\n<|loc0608|><|loc0789|><|loc0913|><|loc0937|>; person\n<|loc0667|><|loc0742|><|loc0844|><|loc0828|>; person\n<|loc0105|><|loc0447|><|loc0891|><|loc0969|>; paddle'}]}]
messages2 = [{'role': 'user', 'content': [{'type': 'image'}, {'type': 'text', 'text': '<|detect|> person;'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '<|loc0082|><|loc0137|><|loc1023|><|loc0857|>; person\n<|loc0537|><|loc0212|><|loc1000|><|loc0440|>; person\n<|loc0608|><|loc0789|><|loc0913|><|loc0937|>; person'}]}]
# Prepare inputs
prompt = processor.apply_chat_template([messages1, messages2])
inputs = processor(text=prompt, images=[im,im], return_tensors="pt",padding=True)

In [70]:
inputs['pixel_values'].shape

torch.Size([2, 17, 3, 512, 512])

In [74]:
inputs['input_ids'].shape, processor.tokenizer.padding_side

(torch.Size([2, 1176]), 'right')

In [71]:
print(processor.tokenizer.decode(inputs['input_ids'].flatten().numpy()))

<|im_start|>User:<fake_token_around_image><row_1_col_1><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><fake_token_around_image><row_1_col_2><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><fake_token

In [65]:
print(processor.chat_template)

<|im_start|>{% for message in messages %}{{message['role'] | capitalize}}{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}{% for line in message['content'] %}{% if line['type'] == 'text' %}{{line['text']}}{% elif line['type'] == 'image' %}{{ '<image>' }}{% endif %}{% endfor %}<end_of_utterance>
{% endfor %}{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}


In [ ]:
def collate_fn_for_completion_only(samples,response_token='<end_of_utterance>'):
    inputs = processor(samples)
    labels = inputs['input_ids'].clone()
    
    assistant_positions = []
    assistant_tokens = processor.tokenizer(assistant_template)['input_ids']

    for batch_idx in range(inputs['input_ids'].shape[0]):
        im_start_positions = torch.where(inputs['input_ids'][batch_idx]==response_token)[0]
        labels[]
        
    for batch_idx in range(inputs['input_ids'].shape[0]):
        inputs['labels'][batch_idx, :assistant_positions[batch_idx]-1] = -100

    return inputs

In [145]:
def collate_fn(examples,response_token='<fake_token_around_image>'):
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    images = [example["image"] for example in examples]
    batch = processor(
        text=texts, 
        images=images, 
        do_rescale=False,
        do_normalize=False,
        return_tensors='pt',
        max_image_size={'longest_edge':512},
        padding=True
    )
    response_token_id = processor.tokenizer.convert_tokens_to_ids(response_token)

    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    batch["labels"] = labels
    
    for batch_idx in range(inputs['input_ids'].shape[0]):
        response_pos = torch.where(inputs['input_ids'][batch_idx]==response_token_id)[0][-1]
        batch['labels'][batch_idx, :response_pos+1] = -100

    return batch

In [140]:
examples = [
    {
        'image': torch.rand(3,512,512),
        'messages': [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": f"<detect> obj1; obj2"}
                ]
            },
            {
                "role": "assitant",
                "content": [
                    {"type": "text", "text": "<loc0000><loc0000><loc0000><loc0000> obj1;\n<loc0000><loc0000><loc0000><loc0000> obj2"}
                ]
            },
        ]
    },
    {
        'image': torch.rand(3,512,512),
        'messages': [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": f"<detect> obj3; obj4"}
                ]
            },
            {
                "role": "assitant",
                "content": [
                    {"type": "text", "text": "<loc0000><loc0000><loc0000><loc0000> obj3;\n<loc0000><loc0000><loc0000><loc0000> obj4;\n<loc0000><loc0000><loc0000><loc0000> obj4"}
                ]
            },
        ]
    }
]

In [146]:
collated = collate_fn(examples)

In [147]:
print(collated['labels'][0].numpy().tolist())

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [148]:
print(processor.tokenizer.decode([t for t in collated['labels'][0].numpy().tolist() if t!=-100]))

<detect> obj1; obj2<end_of_utterance>
Assitant: <loc0000><loc0000><loc0000><loc0000> obj1;
<loc0000><loc0000><loc0000><loc0000> obj2<end_of_utterance>



In [149]:
from trl import SFTConfig, SFTTrainer

In [150]:
collated.keys()

dict_keys(['pixel_values', 'pixel_attention_mask', 'input_ids', 'attention_mask', 'labels'])

In [151]:
from peft import LoraConfig, get_peft_model

In [152]:
model

Idefics3ForConditionalGeneration(
  (model): Idefics3Model(
    (vision_model): Idefics3VisionTransformer(
      (embeddings): Idefics3VisionEmbeddings(
        (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
        (position_embedding): Embedding(1024, 768)
      )
      (encoder): Idefics3Encoder(
        (layers): ModuleList(
          (0-11): 12 x Idefics3EncoderLayer(
            (self_attn): Idefics3VisionAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics3VisionMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=7

In [153]:
targets = ['q_proj','k_proj','v_proj','out_proj','gate_proj','up_proj','down_proj','embed_tokens']

In [154]:
config = LoraConfig(
    r=128,
    lora_alpha=16,
    target_modules=targets,
    lora_dropout=0.1,
    bias="none",
)
model = get_peft_model(model, config)

In [156]:
model.print_trainable_parameters()

trainable params: 50,602,112 || all params: 308,267,840 || trainable%: 16.4150


In [160]:
train_config = SFTConfig(
    output_dir='./VLM_OD',
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    weight_decay=0.25,
    num_train_epochs=1,
    lr_scheduler_type='cosine',
    warmup_ratio=0.2,
    logging_steps=500,
    save_steps=500,
    bf16=True,
    do_eval=False,
    push_to_hub=False,
    report_to='none',
    dataset_num_proc=4,
    save_total_limit=1,
    dataset_kwargs={'skip_prepare_dataset': True},
    remove_unused_columns=False
)